# MLOps : Quand le Machine Learning rencontre l’esprit du DevOps

## Introduction

Les projets de Machine Learning (ML) sont souvent comparés à des expériences scientifiques : on explore, on teste, on ajuste. Une sorte de laboratoire où chaque idée est mise à l’épreuve. Mais cette approche, bien qu’efficace pour apprendre, montre vite ses limites quand il s’agit de passer à la production.

Avec des modèles plus complexes, plus de données, et des besoins de monitoring, il ne suffit plus que « ça marche dans le notebook ». C’est ici que le MLOps, inspiré du DevOps, entre en jeu : il structure, automatise et industrialise chaque étape d’un projet ML. Pour le démontrer, prenons un exemple concret avec un **projet d’analyse de sentiments**. Il s'agit de créer un prototype d'IA pour prédire le sentiment des tweets à partir de données open source et déployé le model via une API Cloud.

## 1) Premier pas

L'une des premières étapes est le choix du modèle, c'est une étape essentielle mais souvent complexe. Cela dépend des spécificités des données, des contraintes du projet et des objectifs visés. Une approche pragmatique consiste à commencer par un modèle simple, puis à explorer des modèles de plus en plus avancés.

Pour notre projet d'analyse de sentiment, nous commençons par **une régression logistique**. Pourquoi ? Parce qu’elle est rapide, facile à implémenter et souvent suffisante pour valider une idée. Avec cette méthode, nous avons utilisé des caractéristiques textuelles de base, comme la fréquence des mots (bag of words). Cette **baseline** joue un double rôle : elle permet de valider rapidement la faisabilité de l'approche et sert de référence pour évaluer les améliorations apportées par des modèles plus complexes.

Une fois que notre régression logistique est en place et que nous avons une baseline, il devient crucial de structurer notre approche pour garantir la reproductibilité des résultats, faciliter la comparaison des expérimentations, et préparer le modèle au déploiement en production, ce que permettent les principes du MLOps et des outils comme **MLflow**. **MLflow** stocke les métadonnées de run et les modèles, tout en assurant le suivi des expérimentations.



En ce qui concerne MLops, nous allons utiliser conjointement plusieurs outils :
- **MLflow**: stocke les métadonnées de run et les modèles.
- **DVC**: stocke et versionne les datasets, sans multiplier l’espace à chaque run. DVC est conçu pour gérer efficacement les changements et éviter de télécharger ou de pousser inutilement des données si elles n'ont pas été modifiées.
- **Git**: stocke les scripts, configurations et fichiers .dvc nécessaires pour synchroniser code et données, assurant une traçabilité complète du projet.
- **Docker/Conda**: Garantie d'avoir un environnement reproductible. 
    - ex : conda pack -n mlops-env -o mlops-env.tar.gz && docker build -f Dockerfile.mlops-env -t mlops-env:v1 .

Ensemble, ces outils créent un écosystème intégré qui garantit la reproductibilité des résultats en synchronisant chaque version du code, des données et des modèles. Ils facilitent également la collaboration entre équipes en offrant un suivi clair des expérimentations et des changements, tout en structurant des pipelines de Machine Learning robustes et facilement déployables à l'échelle.



### Points clés :
- Utilisation de Docker
- L'utilisation de DVC et git permet de faire en sorte que le script 
- l'utilisation d'Hydra permet de changer de passer d'une expérience à autre simplement.

### Résultats :
Voici les performances obtenues sur notre dataset :
- Précision : 76,5%
- F1-score : 74,2%

## Création du pipeline avec DVC (Data Version Control)

Dans un pipeline DevOps classique, vous codez, vous poussez sur Git, et boum, tout se construit et se déploie automatiquement. En MLOps, on va un cran plus loin, les étapes classiques sont : 
- Préparer les données (nettoyage, enrichissement).
- Entraîner les modèles (et tester plusieurs configurations).
- Valider les résultats (cross-validation, ensembles de test).
- Et ensuite ? Surveiller en continu. Parce qu’un modèle, contrairement à du code, peut se dégrader avec le temps !(drift)

En machine learning, les pipelines impliquent non seulement du code, mais aussi des données, des modèles, et des étapes de transformation interconnectées. C'est là que DVC devient un outil indispensable. Inspiré de Git, DVC détecte automatiquement les changements dans votre pipeline, ne réexécutant que les étapes nécessaires pour garantir une gestion efficace et reproductible des workflows ML.


## Monter en puissance avec des modèles avancés

Bien que ce soit un bon point de départ, le modèle manque de finesse pour capturer des relations contextuelles. Pour améliorer les résultats, nous avons besoin de tester des modèles plus avancés, comme des word embeddings combinés à des réseaux neuronaux (CNN ou RNN). Ces approches permettent de mieux comprendre le contexte des mots en tenant compte des séquences.

Avec Hydra, il devient facile d'ajouter des configurations sans trop toucher aux scripts python.

Capture d’écran des résultats et du graphe des erreurs communes.

Résultat avec un CNN :
- Précision : 82,7%
- F1-score : 81,4%

Cette amélioration montre que le modèle capture désormais des nuances plus subtiles dans les sentiments. Cependant, la complexité augmente : le modèle prend plus de temps à entraîner, et le suivi des hyperparamètres devient critique.

Capture d’écran des résultats comparés aux erreurs de la régression logistique.

## Étape 3 : L’État de l’art avec BERT

Enfin, nous avons utilisé BERT, un modèle préentraîné d’État de l’art, capable de comprendre les subtilités linguistiques et les relations complexes dans les phrases. Avec BERT, les résultats ont été impressionnants :

Résultat avec BERT :
- Précision : 89,5%
- F1-score : 88,9%

Cependant, l’utilisation de BERT a introduit de nouveaux défis :
- Temps d’entraînement prolongé : Il a fallu 10 fois plus de temps que pour la régression logistique.
- Infrastructure nécessaire : L’entraînement a nécessité un GPU, et le suivi des versions de données et de modèles est devenu encore plus crucial.

Capture d’écran des résultats, avec une visualisation des prédictions BERT sur des exemples complexes.

## Passage en production avec MLOps

À chaque étape, les modèles deviennent plus performants, mais aussi plus complexes à gérer. C’est ici que le MLOps prend tout son sens. Voici comment il a structuré notre projet :

### 1. Suivi des expérimentations

Grâce à des outils comme MLflow, nous avons enregistré chaque expérimentation :
- Version des données utilisées.
- Hyperparamètres et configurations des modèles.
- Performances obtenues (précision, F1-score, etc.).

Cela nous a permis de comparer les résultats facilement et de justifier nos choix à chaque étape.

### 2. Automatisation des pipelines

Nous avons automatisé les étapes clés :
- Préparation des données (nettoyage, transformation).
- Entraînement des modèles avec différentes configurations.
- Validation sur des ensembles indépendants.

Un outil comme Kubeflow nous a aidés à orchestrer ces tâches et à les rendre reproductibles.

### 3. Monitoring en production

Une fois BERT déployé, un système de monitoring a été mis en place pour surveiller les performances en temps réel. Par exemple :
- Détection des drifts de données : Si les avis clients évoluent (argot, emojis, etc.).
- Suivi des métriques : Si le modèle perd en précision ou en rappel.

Conclusion : Apprendre, structurer, et industrialiser

L’histoire de ce projet d’analyse de sentiments montre bien la double nature des projets ML. D’un côté, il y a l’aspect exploratoire : tester différents modèles et approches pour trouver ce qui fonctionne. De l’autre, il y a le besoin d’industrialisation, où la reproductibilité et la fiabilité deviennent essentielles.

Le MLOps, en s’appuyant sur les principes du DevOps, nous a permis de structurer cette transition. Il a transformé une série d’expérimentations en une solution robuste, prête pour la production. Et à chaque étape, il nous a offert des outils pour gérer la complexité croissante du Machine Learning.